In [17]:
from radartools.datareader import LiveAWR2243Reader
from radartools.radardsp.utils.preprocess.calibration import apply2243_calib_vector_iplab

livereader = LiveAWR2243Reader('/mnt/ssd/409_reflector', nloops=1, nsamples=256, 
                           radarip="192.168.50.253")

## read all captures and frames inside reader directory
# capturenum, _ = reader.count_captures()
# for c in range(capturenum):
#     framenum, _, _ = reader.get_capture_info(c)
#     for f in range(framenum):
#         frame = reader.readframe(c, f)

calibfile = '../resources/id253-hor-cali.mat'

In [18]:
from radartools.radardsp.processor import BeamformingProcessor
from radartools.radardsp.utils.preprocess.antennas import adc2virtual_array

process_setting = '../settings/processing.py'
processor = BeamformingProcessor(process_setting)
# import cProfile

# cProfile.run('processor = BeamformingProcessor(process_setting)')

In [19]:
import scipy.io as sio

rxl, txl = processor.cfg.layout['rxl'], processor.cfg.layout['txl']

cnt = 0
calib_mat = sio.loadmat(calibfile)
# calibrate frame
while True:
    frame = livereader.nextframe()
    if type(frame) == type(None):
        break
    cnt += 1
    calibratated_frame = apply2243_calib_vector_iplab(frame, calib_mat)
    va_adc = adc2virtual_array(calibratated_frame, rxl, txl)
    aoatof = processor.process(va_adc)

In [13]:
print(cnt)

1199


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(abs(aoatof))
plt.gca().invert_yaxis()